# DateSliders in Visualization of Covid-19 cases by Zipcodes with Python/Bokeh

### Import the required modules

In [1]:
import json
import geopandas as gpd
import pandas as pd

from datetime import datetime

from bokeh.io import reset_output, output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Div, Column, Row, ColumnDataSource, FactorRange
from bokeh.models import CustomJS, CustomJSFilter, CDSView, BooleanFilter
from bokeh.models.widgets import DateSlider
from bokeh.models import FuncTickFormatter, FixedTicker

# make bokeh output to notebook
reset_output()
output_notebook()

Loading BokehJS ...

In [2]:
from bokeh.io import reset_output, output_notebook, show
from bokeh.models import RadioButtonGroup
from bokeh.transform import cumsum
from math import pi
from bokeh.palettes import Category10_3
from bokeh.models import LabelSet

In [3]:
# if show_in_notebook is true, then plots will be shown in notebook isnteading of creating standalone HTML.
# This is primarly for debugging/development.
show_in_notebook = False

### Load the Total Count Data 

In [4]:
with open("./Data/Total_Count/all_dates_count_df.json", "r") as f:
    all_dates_count_df = pd.read_json(
        f, convert_dates=["ReportedDate", "UpdatedDatetime"]
    )

In [5]:
all_dates_count_df

,CaseCount,RatePer100000,ReportedDate,TotalCount,UpdatedDatetime,ZipCode
0,629,Unknown,2020-07-31,29577,2020-08-01 08:00:00,Unknown
1,170,468.9,2020-07-31,29577,2020-08-01 08:00:00,92065
2,72,431.6,2020-07-31,29577,2020-08-01 08:00:00,92010
3,94,392.3,2020-07-31,29577,2020-08-01 08:00:00,92011
4,58,410.2,2020-07-31,29577,2020-08-01 08:00:00,92014
...,...,...,...,...,...,...
10360,13,Unknown,2020-04-04,1326,2020-04-05 08:00:00,92173
10361,20,Unknown,2020-04-04,1326,2020-04-05 08:00:00,92139
10362,12,Unknown,2020-04-04,1326,2020-04-05 08:00:00,91942
10363,13,Unknown,2020-04-04,1326,2020-04-05 08:00:00,92064


In [6]:
all_dates_count_df = all_dates_count_df.sort_values(
    by=["ZipCode", "ReportedDate"]
).reset_index(drop=True)
all_dates_count_df["NewCases"] = all_dates_count_df.groupby(["ZipCode"])[
    "CaseCount"
].diff()
all_dates_count_df

,CaseCount,RatePer100000,ReportedDate,TotalCount,UpdatedDatetime,ZipCode,NewCases
0,1,Unknown,2020-04-04,1326,2020-04-05 08:00:00,91901,NaN
1,1,Unknown,2020-04-07,1530,2020-04-08 08:00:00,91901,0.0
2,1,Unknown,2020-04-12,1847,2020-04-13 08:00:00,91901,0.0
3,2,Unknown,2020-04-17,2213,2020-04-18 08:00:00,91901,1.0
4,2,Unknown,2020-04-19,2325,2020-04-20 08:00:00,91901,0.0
...,...,...,...,...,...,...,...
10360,646,Unknown,2020-07-27,28005,2020-07-28 08:00:00,Unknown,8.0
10361,644,Unknown,2020-07-28,28287,2020-07-29 08:00:00,Unknown,-2.0
10362,635,Unknown,2020-07-29,28668,2020-07-30 08:00:00,Unknown,-9.0
10363,606,Unknown,2020-07-30,29048,2020-07-31 08:00:00,Unknown,-29.0


## Plot of total number of cases and new cases with date

In [7]:
total_count_with_date = all_dates_count_df.groupby(['ReportedDate']).first().loc[:,['TotalCount']]
#total_count_with_date = total_count_with_date.asfreq('D')
#total_count_with_date['TotalCount'] =  total_count_with_date['TotalCount'].ffill()
total_count_with_date = total_count_with_date.reset_index()

total_count_with_date['NewCases'] = total_count_with_date['TotalCount'].diff()
total_count_with_date


,ReportedDate,TotalCount,NewCases
0,2020-04-04,1326,NaN
1,2020-04-07,1530,204.0
2,2020-04-12,1847,317.0
3,2020-04-17,2213,366.0
4,2020-04-19,2325,112.0
...,...,...,...
98,2020-07-27,28005,1021.0
99,2020-07-28,28287,282.0
100,2020-07-29,28668,381.0
101,2020-07-30,29048,380.0


In [8]:
total_source = ColumnDataSource(total_count_with_date)
p1 = figure(x_axis_type="datetime", tools = "",
           plot_width=500, plot_height=int(500/1.616),
           )
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Total Number of Cases'
p1.step(x='ReportedDate',y = 'TotalCount', 
        source = total_source,
        color='red', line_width=4,mode='center' )

p2 = figure(x_axis_type="datetime", tools = "",
           plot_width=500, plot_height=int(500/1.616))
p2.grid.grid_line_alpha=0.3
p2.xaxis.axis_label = 'Date'
p2.yaxis.axis_label = 'New Cases'
p2.step(x='ReportedDate',y = 'NewCases', 
        source = total_source,
        color='red', line_width=4,mode='center' )

for p in [p1, p2]:
    p.toolbar_location = None
cases_with_date = Row(p1,p2)

if show_in_notebook:
    show(cases_with_date)

### Get the geojson file of communities

Later in this notebook, I want to plot the data on a map base on zip code geometry. I used the export geojson from https://data.sandiegocounty.gov/Maps-and-Geographical-Resources/Zip-Codes/vsuf-uefy to get the geojson file. In addition to the zip code geometry, it also has the name of the community it belongs to.

In [9]:
county_gpd = gpd.read_file(f'./Data/geojson/Sandiego_Zip_codes.geojson')
county_gpd["x"] = county_gpd.centroid.x
county_gpd["y"] = county_gpd.centroid.y

county_gpd.head()

,community,shape_star,shape_stle,zip,geometry,x,y
0,Alpine,4149939944.16,326045.262676,91901,"POLYGON ((-116.74085 32.95853, -116.74922 32.9...",-116.695575,32.805740
1,Bonita,273909416.836,113257.374615,91902,"POLYGON ((-116.96277 32.70354, -116.96503 32.7...",-117.015050,32.671580
2,Boulevard,2735681408.51,241725.552214,91905,"POLYGON ((-116.23165 32.75083, -116.25284 32.7...",-116.305467,32.718397
3,Campo,3066759065.62,287410.325075,91906,"POLYGON ((-116.35677 32.70460, -116.35870 32.7...",-116.469687,32.660421
4,Chula Vista,403437442.009,112587.791814,91910,"POLYGON ((-117.06354 32.65011, -117.06376 32.6...",-117.065640,32.636404



Merge geojson zip code geometry data with case count data. I do a 'how =right' merge with the zip code as the common key. All the right rows (rows in case count per zip code) will be preserved.

In [10]:
merged_with_geo =county_gpd.merge(all_dates_count_df, right_on = 'ZipCode', left_on = 'zip',
                         how = 'right').drop(columns=['zip']).rename(columns={'community':'CommunityName'})

#df does not have geom information to save sapce
df = merged_with_geo.drop(columns=['geometry', 'shape_star', 'shape_stle']).fillna('Unknown', axis = 0)

## Split the communities into three parts and show bar plots of cases with zipcode.

create a new column to partition Communities into three seperate groups (SanDiego, Others_split1, Others_split2)

In [11]:
df = df.reset_index(drop=True)

for idx, x in enumerate(df['CommunityName']):
    if x == 'San Diego':
        df.at[idx,'split'] = 'San Diego'
    else:
        df.at[idx, 'split'] = 'Others' 

others =df[df['split']=='Others']

for idx, x in others['split'].iteritems():
    if (x=='Others') and (idx<=len(others)//2):
        df.at[idx,'split'] = 'Others_split1'
    elif (x=='Others') and (idx>len(others)//2):
        df.at[idx,'split'] = 'Others_split2'

        
df['zip_community'] = list(zip(df['CommunityName'],df['ZipCode']))

df.loc[:, 'CaseCount_scaled'] = df['CaseCount']/12

In [12]:
source = ColumnDataSource(df)

In [13]:
dates = df['ReportedDate'] 

In [14]:
date_slider = DateSlider(start = dates.min(), end = dates.max(), value = dates.max(), step = 24*60*60*1000)


#This callback is crucial, otherwise the filter will not be triggered when the slider changes
callback = CustomJS(args=dict( date_slider = date_slider,source = source),
                    code="""
                source.change.emit();""")

# Attach the callback to the date slider
date_slider.js_on_change('value', callback)

# Define the custom filter to return the indices from 0 to the desired percentage of total data rows. You could also compare against values in source.data
js_filter = CustomJSFilter(args=dict(date_slider=date_slider),
                           code='''
            var indices = [];
            for (var i = 0; i <= source.data['ReportedDate'].length; i++){
                if (source.data['ReportedDate'][i] == date_slider.value) {
                    indices.push(i)
                }
            }
            return indices
''')


sandiego = CDSView(source=source, filters=[BooleanFilter([True if y_val =='San Diego' else False for y_val in source.data['split']]),js_filter])
others_split1= CDSView(source=source, filters=[BooleanFilter([True if y_val =='Others_split1' else False for y_val in source.data['split']]),js_filter])
others_split2= CDSView(source=source, filters=[BooleanFilter([True if y_val =='Others_split2' else False for y_val in source.data['split']]),js_filter])

sandiego_xrange = list(df.loc[df['split']=='San Diego','zip_community'].unique())
others_split1_xrange = list(df.loc[df['split']=='Others_split1','zip_community'].unique())
others_split2_xrange = list(df.loc[df['split']=='Others_split2','zip_community'].unique())

all_bars = []
for view,xrange in [(sandiego,sandiego_xrange),
             (others_split1,others_split1_xrange),
             (others_split2,others_split2_xrange)
            ]:
    p = figure(plot_width=1000, plot_height=300, x_range=FactorRange(*xrange), 

               toolbar_location=None,tools='',sizing_mode='scale_width',
               tooltips=[("Case Count", "@CaseCount"),
                         ("New Cases", "@NewCases"),
                         ("Community Name, ZipCode", "@zip_community")
                        ]
              )

    p.vbar(x='zip_community', top='CaseCount', width=1, source=source, view = view, line_color="white" )

    p.y_range.start = 0
    p.y_range.end=df['CaseCount'].max()+10
    p.x_range.range_padding = 0.05
    p.xgrid.grid_line_color = None
    p.yaxis.axis_label = "Total Number of Cases"
    p.xaxis.major_label_orientation = 22/28
    p.xaxis.group_label_orientation = 22/28
    p.xaxis.major_label_text_font_size = "7pt"
    p.xaxis.group_text_font_size = "12pt"
    p.title.text_font_size = "16pt"
    p.outline_line_color = None
    p.x_range.group_padding = 1.0
    all_bars.append(p)
bar_case_count= Column(
    Div(text = f'Cases with Zip Code and Community',style={'font-size': '125%', 'color': 'blue'}), 
    date_slider,
    Column(*all_bars))

if show_in_notebook:
    show(bar_case_count)

# Chloropleth Map of Covid 19 Total number cases and New cases according to Zip Code in San Diego

In [15]:
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool

from bokeh.palettes import brewer

geosource = GeoJSONDataSource(geojson= json.dumps(json.loads(county_gpd.to_json())))

rb_cases= RadioButtonGroup(
        labels=["Case Count", "New Cases"], active=0)

source.data['radius'] = source.data['CaseCount_scaled']

map_date_slider = DateSlider(start = dates.min(), end = dates.max(), value = dates.max(), step = 24*60*60*1000)

#unknown = int(df[df['ZipCode']=='Unknown']['CaseCount'].iloc[0])
unknown_text = Div(text = '',style={'font-size': '100%'})


#This callback is crucial, otherwise the filter will not be triggered when the slider changes
callback = CustomJS(args=dict(source = source),
                    code="""
                    source.change.emit();""")

# Attach the callback to the date slider
map_date_slider.js_on_change('value', callback)

# Define the custom filter to return the indices from 0 to the desired percentage of total data rows. You could also compare against values in source.data
map_js_filter = CustomJSFilter(args=dict(unknown_text = unknown_text, date_slider=map_date_slider),
                           code='''
            var indices = [];
            for (var i = 0; i <= source.data['ReportedDate'].length; i++){
                if (source.data['ReportedDate'][i] == date_slider.value) {
                    indices.push(i)
                    if (source.data['ZipCode'][i] == 'Unknown'){
                    var unknown = source.data['CaseCount'][i];
                    unknown_text.text="*Unknown: "+unknown;
                    }
                    
                }
            }
            return indices
''')

map_dateview = CDSView(source=source, filters=[map_js_filter])

#Add hover tool
hover = HoverTool(tooltips = [ ('Total Number of Cases','@CaseCount'),
                              ('New Cases','@NewCases'),
                              ('Zip Code', '@ZipCode'),
                              ('Community Name', '@CommunityName'),
                              ('Reported Date', '@ReportedDate{%F}')
                             ],
                 formatters = {'@ReportedDate':'datetime'},
                 names=['markers'],
                  point_policy='follow_mouse',
                 )
#Create figure object.
map_figure = figure(
    x_axis_location=None, y_axis_location=None,
           plot_height = 500, plot_width = 500, 
           toolbar_location = 'above',
          tools = [hover,'wheel_zoom','pan','reset'],
)

map_figure.xgrid.grid_line_color = None
map_figure.ygrid.grid_line_color = None

#Add patch renderer to figure. 
map_figure.patches('xs','ys', source = geosource,
                   fill_color='white',
#              fill_color = {'field' :'CaseCount', 'transform' : color_mapper},
           line_color = 'black', line_width = 0.25, fill_alpha = 1, name = 'patches')

circles = map_figure.circle(x="x", y="y",size='radius', fill_color="red",
                  line_color='red', fill_alpha=0.5, source=source, view = map_dateview, name = 'markers')

callback_cases = CustomJS(args=dict(rb=rb_cases,circles=circles, source = source),
                    code="""
                        if (rb.active==0){
                            circles.glyph.fill_color.value='red';
                            circles.glyph.line_color.value='red';}
                        else if (rb.active==1){
                                circles.glyph.fill_color.value='blue'
                            circles.glyph.line_color.value='blue';
                            }
                            
                                      
                    for (var i =0; i< source.data['y'].length; i++){
                        if (rb.active==0){
                            source.data['radius'][i] = source.data['CaseCount_scaled'][i];
                            }
                        else if (rb.active==1){
                                if (source.data['NewCases'][i] >= 0){
                                    source.data['radius'][i] = source.data['NewCases'][i];
                                    }
                                else{
                                    source.data['radius'][i] = 0};
                                    }
                            }
                source.change.emit();
                """)
    

# Attach the callback to the radio button group
rb_cases.js_on_change('active', callback_cases)


map_figure.title.text_font_size = '16pt'



cases = Column(Div(text = f'Number of Cases* | New Cases',style={'font-size': '125%', 'color': 'blue'}), 
                              map_date_slider,
               rb_cases,
               map_figure,
               unknown_text)
               

if show_in_notebook:
    show(cases)
    

## Cholorlpleth of Cases per 100000 mapped to ZipCode

In [16]:
dates = df['ReportedDate'].unique()
geosource_df = merged_with_geo[merged_with_geo['ReportedDate'] == dates.max()].drop(columns = ['ReportedDate','UpdatedDatetime']).dropna()
geosource_cases_per_pop = GeoJSONDataSource(geojson= json.dumps(json.loads(geosource_df.to_json())))

chloro_date_slider = DateSlider(start = dates.min(), end = dates.max(), 
                                value = dates.max(), step = 24*60*60*1000)
# Here we plot the chlorpleth of the cases per 10000 cases
#Define a sequential multi-hue color palette.
palette = brewer['YlOrRd'][8]
#
##Reverse color order 
palette = palette[::-1]
#
##Add hover tool
hover = HoverTool(tooltips = [ ('Rate Per 100000','@RatePer100000'),
                              ('Total number of Cases', '@CaseCount'),
                              ('Zip Code', '@ZipCode'),
                              ('Community Name', '@CommunityName'),
                             ],
                 )

#Prepare tick labels for the color bar
max_rateper100000 = df[df['RatePer100000'] != 'Unknown']['RatePer100000'].astype(float).max()-200
#
bins = int(max_rateper100000//7)

if bins%10:
    bins = bins-(bins%10)+10
#
tick_labels = {}
for step in range(0,bins*9,bins):
    tick_labels[step] = str(step)
    
tick_labels[bins*8] = '>'+str(bins*8) 

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, 
                                 high = bins*8 ,nan_color = '#d9d9d9')

formatter=FuncTickFormatter(code =f"""
              var data = {str(tick_labels)};
              return data[tick]
              """)
ticker = FixedTicker(ticks=list(tick_labels.keys()))

#Manual assignment of ticks. 
#color_mapper = LinearColorMapper(palette = palette, low = 0, 
#                                 high = 560)


#formatter=FuncTickFormatter(code ="""
#              var data = {0:"0",70:"70",140:"140",210:"210",280:"280",350:"350",420: "420",490:"490",560:">560"};
#              return data[tick]
#              """)
#ticker = FixedTicker(ticks=[0,70,140,210,280,350,420,490,560])

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, 
                     width = 350, height = 20,border_line_color=None,
                     location = (0,0), 
                     orientation = 'horizontal', 
                     ticker=ticker,
                     formatter=formatter)


chloro_callback = CustomJS(args=dict(source = source, chloro_date_slider=chloro_date_slider,
                                     geosource = geosource_cases_per_pop),
                    code="""
                    geosource.data['RatePer100000'] = [];
                    for (var i = 0; i <= source.data['ReportedDate'].length; i++){
                        if (source.data['ReportedDate'][i] == chloro_date_slider.value) {
                             for (var g = 0 ; g<=geosource.data['x'].length; g++){
                                 if (source.data['x'][i] == geosource.data['x'][g] 
                                 && source.data['y'][i] == geosource.data['y'][g]){
                                 geosource.data['RatePer100000'][g] = source.data['RatePer100000'][i]
                                 geosource.data['CaseCount'][g] = source.data['CaseCount'][i]
                                 }
                             }

                        }
                    }
                   geosource.change.emit(); 
                    """)

# Attach the callback to the date slider
chloro_date_slider.js_on_change('value', chloro_callback)

#Create figure object.
map_cases_per_population = figure(
    x_axis_location=None, y_axis_location=None,
           plot_height = 500,plot_width = 500, 
           toolbar_location = None,
          tools = [hover])


#Add patch renderer to figure. 
map_cases_per_population.patches('xs','ys', source =geosource_cases_per_pop,
          fill_color = {'field' :'RatePer100000', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

map_cases_per_population.add_layout(color_bar, 'below')

cases_per_population_chloro = Column(Div(text = f'Cases per 100000',style={'font-size': '125%', 'color': 'blue'}), 
                              chloro_date_slider,
               map_cases_per_population,)


if show_in_notebook:
    show(cases_per_population_chloro)

## Load the Deaths Data

In [17]:
with open('./Data/Deaths_by_demographics/all_dates_deaths_df.json','r') as f:
    all_dates_deaths_df = pd.read_json(f, convert_dates=['ReportedDate', 'UpdatedDatetime'])
death_dates = all_dates_deaths_df['ReportedDate'].unique()

deaths_df = all_dates_deaths_df[all_dates_deaths_df['ReportedDate']==death_dates.max()]

deaths_by_age = deaths_df[deaths_df['Type']=='Deaths-Age'].set_index('Index').sort_index()
deaths_by_gender = deaths_df[deaths_df['Type']=='Deaths-Gender'].set_index('Index').sort_index()
deaths_by_race = deaths_df[deaths_df['Type']=='Deaths-Race'].set_index('Index').sort_index()

deaths_by_age

,Age Group,Count,Gender,Percent,Race,ReportedDate,Type,UpdatedDatetime
Index,,,,,,,,
0,0-9,0,None,0.0,None,2020-07-31,Deaths-Age,2020-08-01 08:00:00
1,10-19,0,None,0.0,None,2020-07-31,Deaths-Age,2020-08-01 08:00:00
2,20-29,3,None,0.5,None,2020-07-31,Deaths-Age,2020-08-01 08:00:00
3,30-39,5,None,0.9,None,2020-07-31,Deaths-Age,2020-08-01 08:00:00
4,40-49,19,None,3.4,None,2020-07-31,Deaths-Age,2020-08-01 08:00:00
5,50-59,50,None,8.8,None,2020-07-31,Deaths-Age,2020-08-01 08:00:00
6,60-69,98,None,17.3,None,2020-07-31,Deaths-Age,2020-08-01 08:00:00
7,70-79,126,None,22.3,None,2020-07-31,Deaths-Age,2020-08-01 08:00:00
8,80+,264,None,46.7,None,2020-07-31,Deaths-Age,2020-08-01 08:00:00


### Visualize Deaths

In [18]:
deaths_by_age_source = ColumnDataSource(deaths_by_age)
deaths_by_gender_source = ColumnDataSource(deaths_by_gender)
deaths_by_race_source = ColumnDataSource(deaths_by_race)

deaths_by_age_source.data['y'] = deaths_by_age['Count']
deaths_by_gender_source.data['y'] = deaths_by_gender['Count']
deaths_by_race_source.data['y'] = deaths_by_race['Count']


rb_age= RadioButtonGroup(
        labels=["Deaths", "% of Deaths with Known Demographics"], active=0)

#This callback is crucial, otherwise the filter will not be triggered when the slider changes
callback_age = CustomJS(args=dict( rb=rb_age,source = deaths_by_age_source),
                    code="""
                    for (var i =0; i< source.data['y'].length; i++){
                        if (rb.active==0){
                            source.data['y'][i] = source.data['Count'][i]}
                        else if (rb.active==1){
                            source.data['y'][i] = source.data['Percent'][i]}
                        }
                source.change.emit();""")
    

# Attach the callback to the radio button group
rb_age.js_on_change('active', callback_age)

rb_gender= RadioButtonGroup(
        labels=["Deaths", "% of Deaths with Known Demographics"], active=0)

#This callback is crucial, otherwise the filter will not be triggered when the slider changes
callback_gender = CustomJS(args=dict( rb=rb_gender,source = deaths_by_gender_source),
                    code="""
                    for (var i =0; i< source.data['y'].length; i++){
                        if (rb.active==0){
                            source.data['y'][i] = source.data['Count'][i]}
                        else if (rb.active==1){
                            source.data['y'][i] = source.data['Percent'][i]}
                        }
                source.change.emit();""")
    

rb_gender.js_on_change('active', callback_gender)

rb_race= RadioButtonGroup(
        labels=["Deaths", "% of Deaths with Known Demographics"], active=0)

#This callback is crucial, otherwise the filter will not be triggered when the slider changes
callback_race = CustomJS(args=dict( rb=rb_race,source = deaths_by_race_source),
                    code="""
                    for (var i =0; i< source.data['y'].length; i++){
                        if (rb.active==0){
                            source.data['y'][i] = source.data['Count'][i]}
                        else if (rb.active==1){
                            source.data['y'][i] = source.data['Percent'][i]}
                        }
                source.change.emit();""")
    

# Attach the callback to the radio button group
rb_race.js_on_change('active', callback_race)



deaths_by_age_p = figure(x_range=deaths_by_age_source.data['Age Group'],
                         title='Deaths by Age', toolbar_location=None,
                         plot_width=500, plot_height=500, tools='')
labels_by_age = LabelSet(x='Age Group', y='y', text='y', level='glyph',
        x_offset=-1, y_offset=0, source=deaths_by_age_source, render_mode='canvas')

deaths_by_age_p.add_layout(labels_by_age)
deaths_by_age_p.vbar(x = 'Age Group', top = 'y', source=deaths_by_age_source, width=0.9)

deaths_by_gender_p = figure(x_range=deaths_by_gender_source.data['Gender'],
                            plot_width=500, plot_height=500,
                            title="Deaths by Gender", toolbar_location=None,
           )

labels_by_gender = LabelSet(x='Gender', y='y', text='y', level='glyph',
        x_offset=-1, y_offset=0, source=deaths_by_gender_source, render_mode='canvas')
deaths_by_gender_p.vbar(x = 'Gender', top = 'y', source=deaths_by_gender_source, width=0.5)
deaths_by_gender_p.add_layout(labels_by_gender)

deaths_by_race_p = figure(x_range=deaths_by_race_source.data['Race'],
                            plot_width=1000, plot_height=500,
                            title="Deaths by Race/Ethnicity", toolbar_location=None,
           )
    
labels_by_race = LabelSet(x='Race', y='y', text='y', level='glyph',
        x_offset=0, y_offset=0, source=deaths_by_race_source, render_mode='canvas')

deaths_by_race_p.vbar(x = 'Race', top = 'y', source=deaths_by_race_source, width=0.5, alpha=0.9,)

deaths_by_race_p.add_layout(labels_by_race)

for fig in [deaths_by_age_p, deaths_by_gender_p, deaths_by_race_p]:
    fig.yaxis.visible = False
    fig.grid.visible = False
    #fig.xaxis.major_label_orientation =3.14/12

if show_in_notebook:
    show(Column(Row(Column(deaths_by_age_p, rb_age),
         Column(deaths_by_gender_p, rb_gender)),
         Column(deaths_by_race_p, rb_race)))

### Create collage of all individual elements

In [19]:
chloropleth_cases_per_pop = Column( Div(text = 'Cases per 100,000 people',  style={'font-size': '125%', 'color': 'blue'}),
                              map_cases_per_population)
latest_df = df[df['ReportedDate']==dates.max()]
total_count = latest_df.iloc[0]['TotalCount']
reported_date = latest_df.iloc[0]['ReportedDate'].to_pydatetime()
updated_date = latest_df.iloc[0]['UpdatedDatetime'].to_pydatetime()
title = f'Date through {reported_date:%Y-%m-%d}, Updated on {updated_date:%Y-%m-%d %I:%M %p}'
collage = Column(Div(text = title,  style={'font-size': '200%', 'color': 'blue'}),
        Div(text = f'Total number of cases: {total_count}',style={'font-size': '200%', 'color': 'red'}), 
                 cases_with_date,
              Row(cases, cases_per_population_chloro),
                 bar_case_count,
Column(Row(Column(deaths_by_age_p, rb_age),
     Column(deaths_by_gender_p, rb_gender)),
     Column(deaths_by_race_p, rb_race))
                )


                    
                    
#show(cases_map(latest_json_str))

if show_in_notebook:
    show(collage)

### Generate standlone html documents with the collage of both plots

In [20]:
from bokeh.resources import CDN
from bokeh.embed import file_html

if not show_in_notebook:
    try:
        html = file_html(collage, CDN, 'Covid19 Cases in San Diego with Bokeh')
        #with open(f'./HTML_outputs/Covid19_{updated_date:%Y-%m-%d_%I_%M_%p}.html','w') as f:
        with open(f'./HTML_outputs/Covid19.html','w') as f:
            f.write(html)
    except Exception as e:
        print(e)